## Create Data Generating Process (DGP)

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from pykrige.ok import OrdinaryKriging

from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.diagnostics import get_AICc

# Set random seed for reproducibility
np.random.seed(0)

# Generate coordinates for 50 locations for y and x1
num_locations_y_x1 = 1000
coordinates_y_x1 = np.random.rand(num_locations_y_x1, 2) * 100

# Generate coordinates for 50 locations for x2
num_locations_x2 = 500
coordinates_x2 = np.random.rand(num_locations_x2, 2) * 100

# Generate x1 variables (predictor)
X1 = np.random.rand(num_locations_y_x1, 1) * 10

# Generate x2 variables (predictor)
X2 = np.random.rand(num_locations_x2, 1) * 10

# Generate true coefficients for y
true_beta_x1 = 2000
true_beta_x2 = 8000

# Generate noise
noise = np.random.normal(0, 1, num_locations_y_x1)

# Generate y using both x1 and x2
# Interpolate x2 values at y locations (using nearest neighbor for simplicity)

def inverse_distance_weighting(coords_from, values_from, coords_to, power=2):
    tree = cKDTree(coords_from)
    distances, indices = tree.query(coords_to, k=5)
    weights = 1 / distances**power
    weighted_values = np.sum(weights * values_from[indices], axis=1) / np.sum(weights, axis=1)
    return weighted_values



X2_at_y_locations = inverse_distance_weighting(coordinates_x2, X2.flatten(), coordinates_y_x1)


# Generate y
y = (X1.flatten() * true_beta_x1) + (X2_at_y_locations.flatten() * true_beta_x2) + noise


# Create DataFrames to store the data
data_y_x1 = pd.DataFrame({
    'x1': X1.flatten(),
    'y': y,
    'latitude': coordinates_y_x1[:, 0],
    'longitude': coordinates_y_x1[:, 1]
})

data_x2 = pd.DataFrame({
    'x2': X2.flatten(),
    'latitude': coordinates_x2[:, 0],
    'longitude': coordinates_x2[:, 1]
})

- data_y_x1 are on the same support and it represents housing data. Where y is the price, and x1 is the number of bedrooms

- data_x2 represents POI data, and it is simulated at a different support from houses. 

In [2]:
data_y_x1.head()

,x1,y,latitude,longitude
0,4.139625,66825.960707,54.881350,71.518937
1,6.296183,46633.901005,60.276338,54.488318
2,7.785843,38445.756617,42.365480,64.589411
3,8.515578,51941.423003,43.758721,89.177300
4,8.164127,46631.747248,96.366276,38.344152


In [3]:
data_x2.head()

,x2,latitude,longitude
0,2.926420,81.151847,47.608399
1,5.665183,52.315599,25.052059
2,1.374144,60.504302,30.290481
3,3.497122,57.728401,16.967812
4,0.532164,15.946909,41.702974


### Build change of support into GWR from scratch 

<h5> Basic Idea <h5/>
 
- Establish a borrowing threshold (bandwidth) - 10 nearest neighbors

- Select X nearby to each location j of y (with knn). while iterating over each calibration point i

- Using the same threshold, weight each collection of X but use the distances based on i.
  
- Calibrate linear regression on weighted X and weighted y. 

In [4]:
# Define number of neighbors
k_neighbors = 10

# Fit nearest neighbors model for y and x1
nbrs_y_x1 = NearestNeighbors(n_neighbors=k_neighbors).fit(coordinates_y_x1)
distances_y_x1, indices_y_x1 = nbrs_y_x1.kneighbors(coordinates_y_x1)

# Fit nearest neighbors model for x2
nbrs_x2 = NearestNeighbors(n_neighbors=k_neighbors).fit(coordinates_x2)
distances_x2, indices_x2 = nbrs_x2.kneighbors(coordinates_y_x1)

# Gaussian kernel function for weights
def gaussian_kernel(distances, bandwidth):
    return np.exp(-0.5 * (distances / bandwidth) ** 2)

# Define bandwidth
bandwidth = 10

# Compute weights for each location based on distances
weights_y_x1 = gaussian_kernel(distances_y_x1, bandwidth)
weights_x2 = gaussian_kernel(distances_x2, bandwidth)

# Initialize arrays to store smoothed values
X1_smoothed = np.zeros(num_locations_y_x1)
X2_smoothed = np.zeros(num_locations_y_x1)
y_smoothed = np.zeros(num_locations_y_x1)

# Smooth values for each location
for i in range(num_locations_y_x1):
    # Smooth x1 and y using their respective weights
    neighbor_indices_y_x1 = indices_y_x1[i]
    X1_neighbors = X1[neighbor_indices_y_x1].flatten()
    y_neighbors = y[neighbor_indices_y_x1]
    
    W_y_x1 = weights_y_x1[i]
    X1_smoothed[i] = np.average(X1_neighbors, weights=W_y_x1)
    y_smoothed[i] = np.average(y_neighbors, weights=W_y_x1)
    
    # Smooth x2 using its weights
    neighbor_indices_x2 = indices_x2[i]
    X2_neighbors = X2[neighbor_indices_x2].flatten()
    
    W_x2 = weights_x2[i]
    X2_smoothed[i] = np.average(X2_neighbors, weights=W_x2)

# Create a DataFrame to store the smoothed data
smoothed_data = pd.DataFrame({
    'x1_smoothed': X1_smoothed,
    'x2_smoothed': X2_smoothed,
    'y_smoothed': y_smoothed,
    'latitude': coordinates_y_x1[:, 0],
    'longitude': coordinates_y_x1[:, 1]
})

print(smoothed_data.head())

# Perform linear regression on the smoothed data
model = LinearRegression()
model.fit(smoothed_data[['x1_smoothed', 'x2_smoothed']], smoothed_data['y_smoothed'])

# Extract estimated coefficients
estimated_beta = model.coef_
print(f"True coefficients: {true_beta_x1, true_beta_x2}")
print("Estimated coefficients:", estimated_beta)

   x1_smoothed  x2_smoothed    y_smoothed   latitude  longitude
0     4.334394     5.155706  60446.994771  54.881350  71.518937
1     4.933692     4.125080  45002.355035  60.276338  54.488318
2     3.623018     6.626932  64372.565111  42.365480  64.589411
3     5.809252     5.469651  46396.439190  43.758721  89.177300
4     4.527586     4.311152  42373.617671  96.366276  38.344152
True coefficients: (2000, 8000)
Estimated coefficients: [1623.63140549 7816.08057968]


In [5]:
### Calibrate a MGWR model to see if it will recover
### the parameter with just distance weighting

m_X = pd.DataFrame()
m_X['x1'], m_X['x2'] = data_y_x1['x1'], X2_at_y_locations
m_y = data_y_x1['y'].values.reshape((-1,1))
m_X = m_X[[ 'x1', 'x2' ]].values

# m_y = (m_y - m_y.mean(axis=0)) / m_y.std(axis=0)
# m_X = (m_X - m_X.mean(axis=0)) / m_X.std(axis=0)

u = data_y_x1["longitude"]
v = data_y_x1["latitude"]

m_coords = list(zip(u,v))

mgwr_selector = Sel_BW(m_coords, m_y, m_X, multi=True)
mgwr_bw = mgwr_selector.search()
mgwr_results = MGWR(m_coords, m_y, m_X, mgwr_selector).fit()
print('Bandwidth is:', mgwr_bw)
print('=======================')
print('Resid SS is:', mgwr_results.resid_ss)
print('=======================')
print('ENP is:', mgwr_results.ENP)
print('=======================')
print('AICc is:', mgwr_results.aicc)

mgwr_results.summary()

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Bandwidth is: [756. 999. 999.]
Resid SS is: 1002.2462689624128
ENP is: 6.435176186531936
AICc is: 2855.1176697556607
Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           1007.079
Log-likelihood:                                                   -1422.465
AIC:                                                               2850.931
AICc:                                                              2852.971
BIC:                                                              -5879.954
R2:                                                                   1.000
Adj. R2:                                                              1.000

Variable           